<a href="https://colab.research.google.com/github/majiddaeinejad/quera-ai-bootcamp-divar/blob/statistical_analysis/statistical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup cell

In [ ]:
!pip install arabic-reshaper python-bidi jdatetime utm --quiet

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import arabic_reshaper
from bidi.algorithm import get_display
import jdatetime

import geopandas as gpd
from shapely.geometry import Polygon
import folium
from branca.colormap import StepColormap


rcParams['font.family'] = 'DejaVu Sans'

def fa_label(text):
    return get_display(arabic_reshaper.reshape(text))



Load data and exploring


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)


file_path = '/content/drive/MyDrive/Colab Notebooks/Divar.csv'


df = pd.read_csv(file_path , encoding='utf-8')



In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

Missing Values

In [ ]:
total_rows = len(df)

avail = df.notna().sum() / total_rows * 100
missing = df.isna().sum() / total_rows * 100

avail = avail.round(1)
missing = missing.round(1)

nan_status = pd.DataFrame()
nan_status['درصد موجود'] = avail
nan_status['درصد خالی'] = missing

nan_status = nan_status.sort_values(by='درصد موجود', ascending=False)
nan_status



In [ ]:
color_available = "#00BFA5"
color_missing = "#FF8A65"

ax = nan_status.plot(
    kind="barh",
    stacked=True,
    color=[color_available, color_missing],
    figsize=(10,8)
)

plt.xlim(0, 100)
plt.xlabel(fa_label("درصد"))
plt.ylabel(fa_label("ستون‌ها"))
plt.title(fa_label("درصد داده‌های موجود و خالی هر ستون"), fontsize=14)
plt.grid(axis='x', linestyle='--', alpha=0.4)

labels = [fa_label("دادهٔ موجود"), fa_label("دادهٔ خالی")]
plt.legend(labels, title="")

plt.tight_layout()
plt.show()




	توزیع آگهی‌های موجود در دسته‌های مختلف را برای دسته‌بندی سطح دو و سطح سه رسم کنید

value counts

In [ ]:
#category 2
print(df['cat2_slug'].value_counts())

#category 3
print(df['cat3_slug'].value_counts())


cat2

In [ ]:
cat2_labels = {
    "residential-sell": "فروش مسکونی",
    "residential-rent": "اجاره مسکونی",
    "commercial-rent": "اجاره تجاری",
    "commercial-sell": "فروش تجاری",
    "temporary-rent": "اجاره موقت",
    "real-estate-services": "خدمات ملکی"
}


cat2_counts = df['cat2_slug'].value_counts()


colors = plt.cm.Purples(np.linspace(0.4, 0.9, len(cat2_counts)))


plt.figure(figsize=(8, 5))
bars = plt.barh(
    [fa_label(cat2_labels.get(idx, idx)) for idx in cat2_counts.index],
    cat2_counts.values,
    color=colors
)


plt.xlabel(fa_label("تعداد آگهی‌ها"))
plt.ylabel(fa_label("دسته‌بندی سطح ۲"))
plt.title(fa_label("توزیع آگهی‌ها در دسته‌بندی سطح ۲"))
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
cat3_labels = {
    "apartment-sell": "فروش آپارتمان",
    "apartment-rent": "اجاره آپارتمان",
    "plot-old": "زمین/کلنگی",
    "house-villa-sell": "فروش خانه/ویلا",
    "house-villa-rent": "اجاره خانه/ویلا",
    "shop-rent": "اجاره مغازه",
    "shop-sell": "فروش مغازه",
    "office-rent": "اجاره دفتر کار",
    "suite-apartment": "سوئیت آپارتمان",
    "presell": "پیش‌فروش",
    "villa": "ویلا",
    "industry-agriculture-business-sell": "فروش صنعتی/کشاورزی/تجاری",
    "industry-agriculture-business-rent": "اجاره صنعتی/کشاورزی/تجاری",
    "office-sell": "فروش دفتر کار",
    "partnership": "مشارکت",
    "workspace": "فضای کار"
}

cat3_counts = df['cat3_slug'].value_counts()
colors = plt.cm.Purples(np.linspace(0.4, 0.9, len(cat3_counts)))

plt.figure(figsize=(9, 6))
plt.barh(
    [fa_label(cat3_labels.get(i, i)) for i in cat3_counts.index],
    cat3_counts.values,
    color=colors
)
plt.xlabel(fa_label("تعداد آگهی‌ها"))
plt.ylabel(fa_label("دسته‌بندی سطح ۳"))
plt.title(fa_label("توزیع آگهی‌ها در دسته‌بندی سطح ۳"))
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


	هیستوگرام سال ساخت را رسم کنید

In [ ]:
print(df['construction_year'].isna().sum())



In [ ]:
print(df['construction_year'].unique())


In [ ]:
df['construction_year'].value_counts()


In [ ]:
df_clean = df.dropna(subset=['construction_year']).copy()

year_counts = df_clean['construction_year'].value_counts().sort_index()

labels = [fa_label(label) for label in year_counts.index]

plt.figure(figsize=(14, 6))
plt.bar(labels, year_counts.values, color='skyblue', edgecolor='black')
plt.title(fa_label('هیستوگرام سال ساخت'))
plt.xlabel(fa_label('سال ساخت'))
plt.ylabel(fa_label('تعداد'))
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()



In [ ]:
year_counts = df_clean['construction_year'].value_counts().sort_index()
year_percent = (year_counts / year_counts.sum()) * 100


labels = [fa_label(str(label)) for label in year_percent.index]


plt.figure(figsize=(14, 6))
bars = plt.bar(labels, year_percent.values, color='blue', edgecolor='black')


plt.title(fa_label('درصد آگهی های ثبت شده در سال های مختلف'), fontsize=14)
plt.xlabel(fa_label('سال ساخت'), fontsize=12)
plt.ylabel(fa_label('درصد'), fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.5)


for bar, percent in zip(bars, year_percent.values):
    plt.text(bar.get_x() + bar.get_width() / 2,
             bar.get_height() + 0.3,
             f"{percent:.1f}%",
             ha='center', va='bottom', fontsize=10, color='black')

plt.tight_layout()
plt.show()


	تعداد آگهی‌های منتشر شده در ماه‌های مختلف را برای فروش و اجاره بررسی کنید. آیا تعداد آگهی‌های فروش و اجاره در زمان‌های مشخصی از سال افزایش چشم‌گیری داشته است؟

In [ ]:
df_2 = df[df['cat2_slug'] != 'real-estate-services'].copy()

ad_type_mapping = {
    'residential-sell': 'Sale',
    'residential-rent': 'Rent',
    'commercial-rent': 'Rent',
    'commercial-sell': 'Sale',
    'temporary-rent': 'Rent'
}

df_2['ad_type'] = df_2['cat2_slug'].map(ad_type_mapping)

print(df_2['ad_type'].value_counts())



In [ ]:
print(df['created_at_month'].unique())


In [ ]:
df_2['created_at_month'] = pd.to_datetime(df_2['created_at_month'])


In [ ]:
def get_persian_month(date):
    jd = jdatetime.datetime.fromgregorian(datetime=date)
    months = ['فروردین', 'اردیبهشت', 'خرداد', 'تیر', 'مرداد', 'شهریور',
              'مهر', 'آبان', 'آذر', 'دی', 'بهمن', 'اسفند']
    return months[jd.month - 1]


df_2['persian_month'] = df_2['created_at_month'].apply(get_persian_month)


month_order = ['فروردین', 'اردیبهشت', 'خرداد', 'تیر', 'مرداد', 'شهریور',
               'مهر', 'آبان', 'آذر', 'دی', 'بهمن', 'اسفند']
df_2['persian_month'] = pd.Categorical(df_2['persian_month'], categories=month_order, ordered=True)


monthly_counts = df_2.groupby(['persian_month', 'ad_type']).size().unstack(fill_value=0)
monthly_counts


In [ ]:
labels = [get_display(arabic_reshaper.reshape(month)) for month in monthly_counts.index]


monthly_counts.plot(kind='bar', figsize=(10, 5), color=['blue', 'skyblue'])

plt.title(get_display(arabic_reshaper.reshape('تعداد آگهی‌های فروش و اجاره در هر ماه')))
plt.xlabel(get_display(arabic_reshaper.reshape('ماه')))
plt.ylabel(get_display(arabic_reshaper.reshape('تعداد آگهی')))

plt.xticks(ticks=range(len(labels)), labels=labels, rotation=45)

plt.legend([get_display(arabic_reshaper.reshape('فروش')), get_display(arabic_reshaper.reshape('اجاره'))])

plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
change = monthly_counts.pct_change() * 100
significant_months = change[(change > 10).any(axis=1)]
significant_months


In [ ]:
labels = [get_display(arabic_reshaper.reshape(month)) for month in monthly_counts.index]


plt.figure(figsize=(10, 5))
plt.plot(change.index, change['Rent'], marker='o',
         label=get_display(arabic_reshaper.reshape('آگهی اجاره')))
plt.plot(change.index, change['Sale'], marker='o',
         label=get_display(arabic_reshaper.reshape('آگهی فروش')))

plt.axhline(10, color='gray', linestyle='--', alpha=0.5)


plt.title(get_display(arabic_reshaper.reshape('درصد تغییر ماهانه تعداد آگهی‌ها')))
plt.xlabel(get_display(arabic_reshaper.reshape('ماه')))
plt.ylabel(get_display(arabic_reshaper.reshape('درصد تغییر (%)')))


plt.xticks(ticks=range(len(labels)), labels=labels, rotation=45)

plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()



	توزیع قیمت فروش‌(price_value) را برای دسته‌بندی‌های سطح سه در یک نمودار رسم کنید

In [ ]:
df_sales = df_2[df_2['ad_type'] == 'Sale'].copy()



In [ ]:
cat3_labels = {
    "apartment-sell": "فروش آپارتمان",
    "apartment-rent": "اجاره آپارتمان",
    "plot-old": "زمین/کلنگی",
    "house-villa-sell": "فروش خانه/ویلا",
    "house-villa-rent": "اجاره خانه/ویلا",
    "shop-rent": "اجاره مغازه",
    "shop-sell": "فروش مغازه",
    "office-rent": "اجاره دفتر کار",
    "suite-apartment": "سوئیت آپارتمان",
    "presell": "پیش‌فروش",
    "villa": "ویلا",
    "industry-agriculture-business-sell": "فروش صنعتی/کشاورزی/تجاری",
    "industry-agriculture-business-rent": "اجاره صنعتی/کشاورزی/تجاری",
    "office-sell": "فروش دفتر کار",
    "partnership": "مشارکت",
    "workspace": "فضای کار"
}
df_sales['cat3_fa'] = df_sales['cat3_slug'].map(cat3_labels)

In [ ]:
df_sales = df_sales.dropna(subset=['price_value'])

In [ ]:
#تبدیل به میلیارد تومان
df_sales['price_billion'] = df_sales['price_value'] / 1e9

In [ ]:
order = df_sales.groupby('cat3_fa')['price_billion'].median().sort_values().index

In [ ]:
plt.figure(figsize=(12, 8))

sns.boxplot(
    x='price_billion',
    y='cat3_fa',
    data=df_sales,
    order=order,
    palette='Purples'
)

plt.yticks(range(len(order)), [fa_label(c) for c in order])

plt.xlabel(fa_label('قیمت فروش (میلیارد تومان)'))
plt.ylabel(fa_label('دسته‌بندی سطح ۳'))
plt.title(fa_label('توزیع قیمت فروش بر اساس دسته‌بندی سطح ۳'), fontweight='bold')

plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


	بر روی نقشه‌ی جغرافیایی heatmap آگهی‌های هر منطقه را رسم کنید. تراکم آگهی‌ها کدام منطقه بیشتر است؟

Handeling missing values

In [ ]:
print(df[['location_latitude', 'location_longitude']].isna().sum())

In [ ]:
df_filled = df.copy()


neigh_mean = df_filled.groupby('neighborhood_slug')[['location_latitude', 'location_longitude']].transform('mean')
df_filled['location_latitude'] = df_filled['location_latitude'].fillna(neigh_mean['location_latitude'])
df_filled['location_longitude'] = df_filled['location_longitude'].fillna(neigh_mean['location_longitude'])


city_mean = df_filled.groupby('city_slug')[['location_latitude', 'location_longitude']].transform('mean')
df_filled['location_latitude'] = df_filled['location_latitude'].fillna(city_mean['location_latitude'])
df_filled['location_longitude'] = df_filled['location_longitude'].fillna(city_mean['location_longitude'])



In [ ]:
print(df_filled[['location_latitude', 'location_longitude']].isna().sum())

In [ ]:
df_filled = df_filled.dropna(subset=['location_latitude', 'location_longitude'])

In [ ]:
print(df_filled[['location_latitude', 'location_longitude']].isna().sum())

In [ ]:
min_lat, max_lat = df_filled['location_latitude'].min(), df_filled['location_latitude'].max()
min_lon, max_lon = df_filled['location_longitude'].min(), df_filled['location_longitude'].max()


lat_step = 0.05
lon_step = 0.05

polygons, counts, ids = [], [], []
lat_bins = pd.interval_range(min_lat, max_lat, freq=lat_step)
lon_bins = pd.interval_range(min_lon, max_lon, freq=lon_step)
cell_id = 0

for lat_bin in lat_bins:
    for lon_bin in lon_bins:
        count = df_filled[
            (df_filled['location_latitude'].between(lat_bin.left, lat_bin.right)) &
            (df_filled['location_longitude'].between(lon_bin.left, lon_bin.right))
        ].shape[0]
        if count > 0:
            polygons.append(Polygon([
                (lon_bin.left, lat_bin.left),
                (lon_bin.right, lat_bin.left),
                (lon_bin.right, lat_bin.right),
                (lon_bin.left, lat_bin.right)
            ]))
            counts.append(count)
            ids.append(cell_id)
            cell_id += 1

gdf = gpd.GeoDataFrame({'cell_id': ids, 'count': counts, 'geometry': polygons}, crs="EPSG:4326")


max_count = gdf['count'].max()
step_bins = [0, max_count*0.2, max_count*0.4, max_count*0.6, max_count*0.8, max_count]
blue_steps = StepColormap(
    colors=['#deebf7', '#9ecae1', '#6baed6', '#4292c6', '#0868ac', '#08306b'],
    vmin=0, vmax=max_count,
    index=step_bins,
    caption='تعداد آگهی‌ها'
)

m = folium.Map(
    location=[df_filled['location_latitude'].mean(), df_filled['location_longitude'].mean()],
    zoom_start=6
)

folium.GeoJson(
    gdf,
    style_function=lambda feature: {
        'fillColor': blue_steps(feature['properties']['count']),
        'color': 'black',
        'weight': 0.2,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(fields=['count'], aliases=['تعداد آگهی‌ها:'])
).add_to(m)

blue_steps.add_to(m)

m


	ترند میانگین قیمت اجاره بر حسب ماه‌های قرار گرفتن آگهی‌ها رسم کنید.(دقت کنید که ماه‌ها باید به تاریخ شمسی و خوانا باشند.)

In [ ]:
df_rent = df_2[df_2['ad_type'] == 'Rent']


In [ ]:
transformable_rent


In [ ]:
monthly_avg_rent = (
    df_rent.groupby('persian_month')['transformable_rent']
    .mean()
    .reset_index()
)
monthly_avg_rent['rent_million'] = monthly_avg_rent['transformable_rent'] / 1_000_000

plt.figure(figsize=(10, 5))
plt.plot(
    monthly_avg_rent['persian_month'],
    monthly_avg_rent['rent_million'],
    marker='o',
    color='royalblue'
)

plt.title(fa_label('ترند میانگین قیمت اجاره بر حسب ماه‌'), fontsize=14)
plt.xlabel(fa_label('ماه '))
plt.ylabel(fa_label('میانگین اجاره (میلیون تومان)'))

plt.xticks(
    ticks=range(len(monthly_avg_rent)),
    labels=[fa_label(m) for m in monthly_avg_rent['persian_month']]
)

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

